1. Data Loader
2. Merge CSV into one master
3. Remove to return the cheapest flight??
4. Clean Data
5. Feature Engineering

In [2]:
import pandas as pd
import numpy as np
import zipfile 
import io
import os
import concurrent.futures
from concurrent.futures import ProcessPoolExecutor
import time
import subprocess
import multiprocessing
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from sklearn.model_selection import train_test_split


# Data Loader

## Function to unzip folder

In [2]:
# Extract the zip and save in interim
data_folder = '../data'
zip_flight_data= data_folder + '/external/itineraries_csv.zip'

extract_directory = data_folder +'/interim'

# Read the zip file
with zipfile.ZipFile(zip_flight_data, 'r') as z:
    z.extractall(extract_directory)

print(f"Extracted {zip_flight_data} to {extract_directory}")



Extracted ../data/external/itineraries_csv.zip to ../data/interim


## Function to unzip CSV and save them as 1 per airport

In [3]:
# CHAMPION FILE EXTRACTOR
def process_folder(folder):    
    df_list = []
    files = os.listdir(f"../data/interim/itineraries_csv/{folder}")
    for file in files:
        file_path = f"../data/interim/itineraries_csv/{folder}/{file}"
        if file.endswith('.zip'):
            df=pd.read_csv(file_path, compression='zip')
            df_list.append(df)
    df_list = pd.concat(df_list)
    df_list.to_csv(f"../data/interim/{folder}.csv", index=False)
    print(f"Saved {folder}.csv")
root_dir = "../data/interim/itineraries_csv"
folders = [entry.name for entry in os.scandir(root_dir) if entry.is_dir()]

# Measure execution time
start_time = time.time()

# Limit the number of threads
max_threads = 4

# Use multiprocessing.Pool to run the folder processing function in parallel
with multiprocessing.Pool(processes=max_threads) as pool:
    pool.map(process_folder, folders)

end_time = time.time()
elapsed_time = end_time - start_time
print(f"Execution time: {elapsed_time:.2f} seconds")

KeyboardInterrupt: 

In [9]:
# Remove the processed folders to save storage space
subprocess.run(f"rm -rf ../data/interim/itineraries_csv", shell=True)
subprocess.run(f"rm -rf ../data/interim/__MACOSX", shell=True)
print("Removed folders")


Removed folders


# Transform

## Clean Data

In [2]:
# Function to clean and convert the data
def clean_data(df):

    # Convert date columns
    date_columns = ['searchDate', 'flightDate']
    df[date_columns] = df[date_columns].apply(pd.to_datetime)

    # Downcast float columns
    float_columns = ['totalFare','totalTravelDistance']
    for col in float_columns:
        df[col] = pd.to_numeric(df[col],errors='coerce').astype('float32')

    # Drop columns
    drop_columns = ['legId', 'traveDuration','segmentsDepartureTimeEpochSeconds', 'segmentsArrivalTimeEpochSeconds','segmentsArrivalTimeRaw', 'segmentsDepartureAirportCode','segmentsAirlineCode','segmentsEquipmentDescription','segmentsDurationInSeconds', 'segmentsDistance']
    return df

    


## Feature Engineering

### Segments

In [3]:
# Convert the segment data into new variables

airports = ['ATL', 'BOS', 'DEN', 'DFW', 'EWR', 'JFK', 'LAX', 'LGA', 'MIA', 'OAK', 'ORD', 'PHL', 'SFO']
# Dictionary for the correct time offsets
time_offsets = {'ATL': '-04:00', 'BOS': '-04:00', 'DEN': '-06:00', 'DFW': '-05:00', 'EWR': '-04:00', 'JFK': '-04:00', 'LAX': '-07:00', 'LGA': '-04:00', 'MIA': '-04:00', 'OAK': '-07:00', 'ORD': '-05:00', 'PHL': '-04:00', 'SFO': '-07:00'}

# Remove row with more than 2 segments
def process_segments_stops(df):
    # Create boolean filter
    filter = df["segmentsCabinCode"].str.count("\\|\\|") <= 1
    df = df[filter]
    return df

# Departure Hour Processing
def process_segments_DepartTime(df):
    # Split the departure time
    print("Shape before processing: ", df.shape)
    df["segmentsDepartureTimeRaw_hour"] = df["segmentsDepartureTimeRaw"].str.split("|").str[0]
    airportcode = df["startingAirport"].iloc[0]
    correct_time_offset = time_offsets[airportcode]

    # Filter for departure time hour that has the correct time offset
    df = df[df["segmentsDepartureTimeRaw_hour"].str[-6:] == correct_time_offset]
    print("Shape after processing departure time: ", df.shape)

    # Check date difference between the segmentsDepartureTimeRaw_hour and flight date and eliminate any that don't match
    df["segmentsDepartureTimeRaw_hour"] = pd.to_datetime(df["segmentsDepartureTimeRaw_hour"]).dt.tz_localize(None)
    df = df[(df["segmentsDepartureTimeRaw_hour"] - df["flightDate"]).dt.days == 0]
    print("Shape after filtering segment departure not the same as flight date: ", df.shape)

    # Extract out the hour of the day
    df["DepartureTimeHour"] = df["segmentsDepartureTimeRaw_hour"].dt.hour.astype("uint8")

    return df

# Cabin Class Encoding
cabin_class_encoding = {
    "coach": 1,
    "premium coach": 2,
    "business": 3,
    "first": 4,
}

def process_segments_CabinClass(df):
    # Split the Cabin class
    df["CabinCode"] = df["segmentsCabinCode"].str.split("\\|\\|")
    
    # Apply cabin_class_encoding on the cabin class
    df["CabinCode"] = df["CabinCode"].apply(lambda x: [cabin_class_encoding.get(i, 0) for i in x])

    # Get the average of the cabin class
    df["CabinCode"] = df["CabinCode"].apply(lambda x: np.mean(x) if isinstance(x, list) else x).astype("float32")

    return df

# Airline Name Processing

# Airline Encoding 
# 1: Ultra Low Cost, 2: Budget, 3: Regional, 4: Full Service
airline_type_encoding = {
    'JetBlue Airways': 2,
    'Sun Country Airlines': 2,
    'United': 4,
    'Delta': 4,
    'Key Lime Air': 3,
    'Boutique Air': 3,
    'Contour Airlines': 3,
    'Spirit Airlines': 1,
    'American Airlines': 4,
    'Alaska Airlines': 4,
    'Southern Airways Express': 3,
    'Frontier Airlines': 1,
    'Hawaiian Airlines': 4,
    'Cape Air': 3
}

# Airline Category Mapping
def process_segments_AirlineNameScore(df):
    df["AirlineNameScore"] = df["segmentsAirlineName"].str.split("\\|\\|")
    df["AirlineNameScore"] = df["AirlineNameScore"].apply(lambda x: [airline_type_encoding.get(i, 0) for i in x])
    df["AirlineNameScore"] = df["AirlineNameScore"].apply(lambda x: np.max(x) if isinstance(x, list) else x).astype("uint8") 
    return df

# 

### Date

In [4]:
def add_time_difference_columns(df):
    # Calculate the difference in days
    df['date_diff_days'] = (df["flightDate"] - df["searchDate"]).dt.days.astype('uint16')

    # Calculate sin, cos to train cyclical patterns
    # Day of the week
    df['weekday'] = df["flightDate"].dt.weekday.astype('uint8')
    # df['weekday_sin'] = np.sin(2 * np.pi * df['weekday'] / 7).astype('float32')
    # df['weekday_cos'] = np.cos(2 * np.pi * df['weekday'] / 7).astype('float32')
    
    return df

# Keep only the useful columns and cheapest flight rows
!!!!!!!!!!!!!MAKE SURE YOU UPDATE IF NEW VARIBLES CREATED!!!!!!!!!!!!!!

In [5]:
keep_columns=['searchDate', 'flightDate', 'startingAirport', 'destinationAirport', 'isNonStop', 'isRefundable', 'isBasicEconomy', 'totalFare', 'totalTravelDistance', 'segmentsArrivalAirportCode','DepartureTimeHour', 'CabinCode', 'AirlineNameScore', 'date_diff_days', 'weekday' ] #'weekday_sin', 'weekday_cos'
def keep_essential_columns(df):
    return df[keep_columns]

# Keep cheapess flight
# Remove row so only the cheapest for the feature set is kept
features_column = ['searchDate', 'flightDate', 'startingAirport', 'destinationAirport', 'isNonStop', 'isRefundable', 'isBasicEconomy','DepartureTimeHour', 'CabinCode', 'AirlineNameScore', 'date_diff_days']

def keep_cheapest_flight(df):
    df = df.sort_values('totalFare').drop_duplicates(features_column).sort_index()
    return df


In [3]:
df = pd.read_csv('../data/interim/SFO.csv')
# df_clean = clean_data(df)

# df_clean = process_segments_stops(df_clean)

# df_cheapest = process_data(df)


## Run all functions

In [6]:
def process_data(df):
    df = clean_data(df)
    df = process_segments_stops(df)
    df = process_segments_DepartTime(df)
    df = process_segments_CabinClass(df)
    df = process_segments_AirlineNameScore(df)
    df = add_time_difference_columns(df)
    df = keep_essential_columns(df)
    df = keep_cheapest_flight(df)
    return df

In [8]:
df_processed=process_data(df)

Shape before processing:  (871561, 23)


/tmp/ipykernel_222478/203945261.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["segmentsDepartureTimeRaw_hour"] = df["segmentsDepartureTimeRaw"].str.split("|").str[0]


Shape after processing departure time:  (871561, 24)
Shape after filtering segment departure not the same as flight date:  (871561, 24)


In [9]:
df_processed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 494381 entries, 0 to 949045
Data columns (total 15 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   searchDate                  494381 non-null  datetime64[ns]
 1   flightDate                  494381 non-null  datetime64[ns]
 2   startingAirport             494381 non-null  object        
 3   destinationAirport          494381 non-null  object        
 4   isNonStop                   494381 non-null  bool          
 5   isRefundable                494381 non-null  bool          
 6   isBasicEconomy              494381 non-null  bool          
 7   totalFare                   494381 non-null  float32       
 8   totalTravelDistance         490401 non-null  float32       
 9   segmentsArrivalAirportCode  494381 non-null  object        
 10  DepartureTimeHour           494381 non-null  uint8         
 11  CabinCode                   494381 non-null 

In [ ]:
df_processed.info()

<class 'pandas.core.frame.DataFrame'>
Index: 874055 entries, 0 to 874068
Data columns (total 30 columns):
 #   Column                             Non-Null Count   Dtype         
---  ------                             --------------   -----         
 0   legId                              874055 non-null  object        
 1   searchDate                         874055 non-null  datetime64[ns]
 2   flightDate                         874055 non-null  datetime64[ns]
 3   startingAirport                    874055 non-null  object        
 4   destinationAirport                 874055 non-null  object        
 5   travelDuration                     874055 non-null  object        
 6   isBasicEconomy                     874055 non-null  bool          
 7   isRefundable                       874055 non-null  bool          
 8   isNonStop                          874055 non-null  bool          
 9   totalFare                          874055 non-null  float32       
 10  totalTravelDistance      

In [ ]:
# size of the df

# Merge into one master


In [7]:
def process_csv(file_path):
    df = pd.read_csv(file_path)
    df = process_data(df)
    return df

def process_and_combine_csv_parallel(directory, airports, max_workers=6):
    start_time = time.time()
    master_df = pd.DataFrame()
    file_paths = [os.path.join(directory, f'{airport}.csv') for airport in airports if os.path.exists(os.path.join(directory, f'{airport}.csv'))]
    # Option 1: Use concurrent.futures.ProcessPoolExecutor
    with ProcessPoolExecutor(max_workers=max_workers) as executor:
        results = executor.map(process_csv, file_paths)
    
    for df in results:
        master_df = pd.concat([master_df, df], ignore_index=True)
    
    # # Option 2: Use multiprocessing.Pool
    # with multiprocessing.Pool(processes=6) as pool:
    #     results = pool.map(process_csv, file_paths)
    
    # for df in results:
    #     master_df = pd.concat([master_df, df], ignore_index=True)

    end_time = time.time()
    print(f"Execution time: {end_time - start_time:.2f} seconds")
    return master_df

In [8]:
# 6 workers
directory = '../data/interim'

master_df = process_and_combine_csv_parallel(directory, airports, max_workers=6) # Adjust workers if sytem crashes due to memory


Shape before processing:  (647768, 23)
Shape before processing:  (685158, 23)
Shape before processing:  (749151, 23)


/tmp/ipykernel_228931/203945261.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["segmentsDepartureTimeRaw_hour"] = df["segmentsDepartureTimeRaw"].str.split("|").str[0]
/tmp/ipykernel_228931/203945261.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["segmentsDepartureTimeRaw_hour"] = df["segmentsDepartureTimeRaw"].str.split("|").str[0]


Shape before processing:  (850459, 23)
Shape before processing:  (828482, 23)
Shape after processing departure time:  (647768, 24)
Shape after processing departure time:  (685158, 24)


/tmp/ipykernel_228931/203945261.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["segmentsDepartureTimeRaw_hour"] = df["segmentsDepartureTimeRaw"].str.split("|").str[0]


Shape before processing:  (958591, 23)
Shape after filtering segment departure not the same as flight date:  (647768, 24)
Shape after filtering segment departure not the same as flight date:  (685158, 24)
Shape after processing departure time:  (749151, 24)


...

Shape after filtering segment departure not the same as flight date:  (826259, 24)


/tmp/ipykernel_228931/203945261.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["segmentsDepartureTimeRaw_hour"] = df["segmentsDepartureTimeRaw"].str.split("|").str[0]
/tmp/ipykernel_228931/203945261.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["segmentsDepartureTimeRaw_hour"] = df["segmentsDepartureTimeRaw"].str.split("|").str[0]
/tmp/ipykernel_228931/203945261.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

Shape after processing departure time:  (1306695, 24)
Shape after processing departure time:  (864568, 24)
Shape after processing departure time:  (734346, 24)
Shape after filtering segment departure not the same as flight date:  (1306695, 24)
Shape after filtering segment departure not the same as flight date:  (864568, 24)
Shape after filtering segment departure not the same as flight date:  (734346, 24)
Shape before processing:  (871561, 23)


/tmp/ipykernel_228931/203945261.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["segmentsDepartureTimeRaw_hour"] = df["segmentsDepartureTimeRaw"].str.split("|").str[0]


Shape after processing departure time:  (871561, 24)
Shape after filtering segment departure not the same as flight date:  (871561, 24)
Execution time: 108.67 seconds


In [9]:
# Define the features and target
X = master_df.drop(['totalFare'], axis=1)
y = master_df['totalFare']

# Split data into 80:20 for training and testing
train_data, test_data = train_test_split(master_df, test_size=0.2, random_state=42)

# Save the data
train_data.to_feather('../data/processed/train_data.feather')
test_data.to_feather('../data/processed/test_data.feather')

# # Open the data
# pd.read_feather('../data/processed/train_data.feather')
# pd.read_feather('../data/processed/test_data.feather')

In [10]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5287832 entries, 6164974 to 6413414
Data columns (total 15 columns):
 #   Column                      Dtype         
---  ------                      -----         
 0   searchDate                  datetime64[ns]
 1   flightDate                  datetime64[ns]
 2   startingAirport             object        
 3   destinationAirport          object        
 4   isNonStop                   bool          
 5   isRefundable                bool          
 6   isBasicEconomy              bool          
 7   totalFare                   float32       
 8   totalTravelDistance         float32       
 9   segmentsArrivalAirportCode  object        
 10  DepartureTimeHour           uint8         
 11  CabinCode                   float32       
 12  AirlineNameScore            uint8         
 13  date_diff_days              uint16        
 14  weekday                     uint8         
dtypes: bool(3), datetime64[ns](2), float32(3), object(3), uint16(1), 

In [ ]:
# master_df.to_feather('../data/processed/master.feather')
# Open the feather file
# df = pd.read_feather('../data/processed/master.feather')
# df.info()

